In [2]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt

In [10]:
m = pyo.ConcreteModel()

m.exset1 = pyo.RangeSet(1,3)
m.exset2 = pyo.RangeSet(2,6,2)
#m.expar2 = pyo.Param(m.exset1, initialize=m.exset2)
#pyo.value(m.expar2[1])
exdict = {pyo.value(i)+1: pyo.value(i) for i in m.exset2}
m.N = pyo.Param(initialize=5, mutable=True)
m.J = pyo.RangeSet(1,m.N*2+3)
m.N = 4
Jdict = {i: i for i in m.J}
Jdict
Jarr = [i for i in m.J]
Jarr


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [11]:
def build_model(sbounds=1, N=1):
    '''
    Builds farmland allocation model
    
    Inputs:
    sbounds: crop yield scenario bounds as fractions of average yield
    N: number of scenarios
    '''
    # Declare model
    m = pyo.ConcreteModel()
    
    if type(sbounds) == 'int' or type(sbounds) == 'float':
        sbounds = (sbounds,sbounds)
    
    if len(sbounds) == 1:
        sbounds = (sbounds[0],sbounds[0])
    # Verify that input arguments are valid
    assert sbounds[0] > 0, "Scenario bounds cannot be negative."
    if len(sbounds) > 2:
        m.N = pyo.Param(initialize=len(sbounds))
        m.J = pyo.RangeSet(1,m.N)
        m.Sfrac = pyo.Param(m.J,initialize={i+1:sbounds[i] for i in range(N)})
    else:
        assert N%1 == 0 and N > 0, "Number of scenarios N must be a positive integer."
        if sbounds[0] == sbounds[1]:
            assert N==1, "N must be 1 if scenario bounds are not specified."
        else:
            assert N > 1, "N must be greater than 1 if two different scenario bounds are specified."
            assert sbounds[0] < sbounds[1], "Second scenario bound must be greater than first scenario bound."
        m.N = pyo.Param(initialize=N)
        m.J = pyo.RangeSet(1,m.N)
        m.sbounds = pyo.Param([0,1], initialize={0:sbounds[0], 1:sbounds[1]})
        if N > 1:
            m.Sfracset = pyo.RangeSet(m.sbounds[0], m.sbounds[1]+(m.sbounds[1]-m.sbounds[0])/(m.N-1)*0.5, (m.sbounds[1]-m.sbounds[0])/(m.N-1))
        else:
            m.Sfracset = pyo.Set(initialize=pyo.value(m.sbounds[0]))
        m.Sfrac = pyo.Param(m.J,initialize={i:pyo.value(m.Sfracset[i]) for i in m.J})
    
    m.xind = pyo.RangeSet(1,3)
    m.yind = pyo.RangeSet(1,2)
    m.wind = pyo.RangeSet(1,4)
    
    # Define additional parameters
    m.Px = pyo.Param(m.xind, initialize={1:150, 2:230, 3:260})
    m.Py = pyo.Param(m.yind, initialize={1:238, 2:210})
    m.Pw = pyo.Param(m.wind, initialize={1:170, 2:150, 3:36, 4:10})
    m.totalarea = pyo.Param(initialize=500)
    m.quota = pyo.Param(initialize=6000)
    m.baseyield = pyo.Param(m.xind,initialize={1:2.5, 2:3.0, 3:20.0})
    m.Feed = pyo.Param(m.yind, initialize={1:200,2:240})
    m.weight = pyo.Param(initialize=1.0/m.N)
    
    # Define variables
    m.x = pyo.Var(m.xind, within=pyo.NonNegativeReals)
    m.y = pyo.Var(m.yind, m.J, within=pyo.NonNegativeReals)
    m.w = pyo.Var(m.wind, m.J, within=pyo.NonNegativeReals)
    
    # Define objective
    @m.Objective()
    def costobj(m):
        return sum(m.Px[i]*m.x[i] for i in m.xind) + m.weight*sum(sum(m.Py[i]*m.y[i][j] for i in m.yind)-sum(m.Pw[i]*m.w[i][j] for i in m.wind) for j in m.J)
    
    # Define constraints
    # mass balance on wheat and corn
    @m.Constraint(m.yind, m.J)
    def feedconsumption(m, i, j):
        return m.Feed[i] == m.baseyield[i]*m.Sfrac[j]*m.x[i] + m.y[i][j] - m.w[i][j]
    
    # mass balance on beets
    @m.Constraint(m.J)
    def beetmb(m, j):
        return m.baseyield[3]*m.Sfrac[j]*m.x[3] == m.w[3][j] + m.w[4][j]
    
    # beet quota constraint
    @m.Constraint(m.J)
    def quotaconstraint(m, j):
        return m.w[3][j] <= m.quota
    
    # farmland availability
    @m.Constraint()
    def farmlandconstraint(m):
        return sum(m.x[i] for i in m.xind) <= m.totalarea
    
    return m
    
    
    

In [ ]:
def get_results(m, detailed_WS=False):
    '''
    '''
    
    cropdict = {1: "Wheat", 2:"Corn", 3:"Sugar Beets"}
    
    solver = pyo.SolverFactory('ipopt')
    
    results1 = solver.solve(m)
    
    x_RP = pd.Series({cropdict[i]:pyo.value(m.x[i]) for i in m.xind})
    
    N = pyo.value(m.N)
    plant_PI_df = pd.DataFrame()
    profit_PI = np.zeros(N)
    for j in m.J:
        sreal = pyo.value(m.Sfrac[j])
        m_PI = build_model(sreal)
        results_PI = solver.solve(m_PI)
        profit_PI[j-1] = -pyo.value(m_PI.costobj)
        if detailed_WS:
            plant_PI_df["{:+.0f}%".format((sreal-1)*100.0)] = pd.Series({cropdict[i]:m_PI.x[i] for i in m_PI.xind})
    WS = np.mean(profit_PI)
    EV = np.mean(np.array([pyo.value(m.Sfrac[j]) for j in m.J]))
    m_EV = build_model(EV)
    results_EV = solver.solve(m_EV)
    for i in m.xind:
        m.x[i].fix(pyo.value(m_EV.x[i]))
    results_EEV = solver.solve(m)
    EEV = pyo.value(m.costobj)
    
    
    return profit, planted